In [37]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import copy
from PIL import Image
from io import BytesIO

import subprocess
import sys
import re

sample_ply_data = o3d.data.DemoCropPointCloud()
tst_pointCloud = o3d.io.read_point_cloud(sample_ply_data.point_cloud_path)
#o3d.visualization.draw_geometries([tst_pointCloud])

def array_to_data(array):
    im = Image.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.degrees(RX), np.degrees(RY), np.degrees((RZ))))
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color

def Convert_message(string):
    return [int(s) for s in re.findall('[0-9]', string)]



In [ ]:
clouds = [tst_pointCloud,tst_pointCloud]

In [41]:
def Cluster_selection_gui(clouds):
    dat=[]  #to hold cluster's images
    pcd_combined = o3d.geometry.PointCloud()
    
    for current_cloud in clouds:
            color = fetch_cloud_image(current_cloud, RX=-45)  #different point clouds to be viewed
            dat.append(color)
    result = subprocess.run([sys.executable,  "Cluster_selection_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(dat))
    selected_PC = Convert_message(result.stdout)

    for k in selected_PC:
        pcd_combined += clouds[k]  #combine selected pointclouds 
    return pcd_combined



In [43]:
Cluster_selection_gui(clouds)

PointCloud with 392266 points.